# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import pickle
import re
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Fujitsu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Fujitsu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Fujitsu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import warnings
# ignore warnings
warnings.filterwarnings("ignore")

In [3]:
# load data from database
engine = create_engine('sqlite:///data_disaster_prepared.db')
df = pd.read_sql_table('messages',con=engine)
X = df['message']
y = df[df.columns[5:]]

### 2. Write a tokenization function to process your text data

In [4]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
stop_words = stopwords.words("english")

def tokenize(text, lemmatizer=WordNetLemmatizer()):
    # Detecte URLs
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')
    
    # Normalize and tokenize
    tokens = nltk.word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower()))
    
    # Remove stopwords
    tokens = [t for t in tokens if t not in stop_words]

    # Lemmatize
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
#Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)
#Train pipeline
pipeline.fit(X_train.as_matrix(), y_train.as_matrix())
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# Get results and add them to a dataframe.
def test_model(y_test, y_pred):
    
    test_model = pd.DataFrame(columns=['Category','precision', 'recall','f_score'])
    
    i = 0
    for cat in y_test.columns:
        precision, recall, f_score, support = precision_recall_fscore_support(y_test[cat], y_pred[:,i], average='weighted')
        test_model.set_value(i+1, 'Category', cat)
        test_model.set_value(i+1, 'precision', precision)
        test_model.set_value(i+1, 'recall', recall)
        test_model.set_value(i+1, 'f_score', f_score)
        i += 1
        
    print('f_score:', test_model['f_score'].mean())
    print('recall:', test_model['recall'].mean())
    print('precision:', test_model['precision'].mean())
    
    return test_model

In [8]:
test_model(y_test, y_pred)

f_score: 0.9366896946978959
recall: 0.948690003923449
precision: 0.937731200361922


,Category,precision,recall,f_score
1,request,0.87961,0.886939,0.872398
2,offer,0.99117,0.995575,0.993368
3,aid_related,0.750844,0.751449,0.745919
4,medical_help,0.901932,0.923253,0.897803
5,medical_products,0.941312,0.953006,0.935322
6,search_and_rescue,0.957623,0.970705,0.958903
7,security,0.975699,0.983522,0.975649
8,military,0.955906,0.965517,0.951765
9,child_alone,1,1,1
10,water,0.94548,0.951022,0.941951


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x00000239F8EEC488>,
           vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
               oob_score=Fal

In [10]:
parameters = {'clf__estimator__max_depth': [2, 3, 5, 10],
              'clf__estimator__min_samples_leaf':[1,5,8]}

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
cv.fit(X_train.as_matrix(), y_train.as_matrix())
y_pred = cv.predict(X_test)
test_model(y_test, y_pred)

f_score: 0.9008541408454709
recall: 0.9316142813548979
precision: 0.8842530978883953


,Category,precision,recall,f_score
1,request,0.694021,0.833079,0.757219
2,offer,0.99117,0.995575,0.993368
3,aid_related,0.675822,0.589258,0.438485
4,medical_help,0.847051,0.920354,0.882183
5,medical_products,0.903863,0.950717,0.926698
6,search_and_rescue,0.942564,0.970857,0.956502
7,security,0.967315,0.983522,0.975351
8,military,0.929868,0.964297,0.946769
9,child_alone,1,1,1
10,water,0.870238,0.932865,0.900464


In [12]:
cv.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
# testing with a decision tree classifier

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))])


In [14]:
#Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)
# Train pipeline
pipeline.fit(X_train.as_matrix(), y_train.as_matrix())
y_pred = pipeline.predict(X_test)

In [15]:
test_model(y_test, y_pred)

f_score: 0.9379813322877971
recall: 0.9385936614499323
precision: 0.9374693954500336


,Category,precision,recall,f_score
1,request,0.858883,0.859323,0.859101
2,offer,0.989938,0.991456,0.990696
3,aid_related,0.710933,0.710558,0.710738
4,medical_help,0.900468,0.905554,0.902896
5,medical_products,0.943098,0.944156,0.943619
6,search_and_rescue,0.962087,0.961855,0.961971
7,security,0.969087,0.971773,0.970404
8,military,0.961834,0.961398,0.961614
9,child_alone,1,1,1
10,water,0.957708,0.954532,0.955929


### 9. Export your model as a pickle file

In [16]:
pickle.dump(cv, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.